In [1]:
import create_data
import kmeans
import voltage
import os
import importlib
import time
import bpf
import numpy as np


from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [2]:
importlib.reload(create_data)
importlib.reload(kmeans)
importlib.reload(voltage)
importlib.reload(bpf)

<module 'bpf' from 'C:\\Users\\avigh\\Documents\\python\\VoltageDimentionalReduction\\code\\bpf.py'>

In [3]:
print("Loading Data...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(np.int64)

# data = create_data.Data(np.array(X))

Loading Data...


In [4]:
subDivision = {}
summation = {}
count = {}

print("Sorting and averaging...")

for xi, yi in zip(X, y):
    if yi in summation:
        subDivision[yi].append(np.array(xi))
        summation[yi] += np.array(xi)
        count[yi] += 1
    else:
        subDivision[yi] = [np.array(xi)]
        summation[yi] = np.array(xi)
        count[yi] = 1

Sorting and averaging...


In [5]:
print("Kmeans...")

k = 10
landmarks = []
data = []

for yi in range(10):
    print(yi)
    
    partitions = kmeans.Partitions(subDivision[yi])
    partitions.k_means(k, seed=time.time())

    data += list(partitions.centers)

for yi in range(10):
    landmarks.append(voltage.createLandmarkClosestTo(data, summation[yi] / count[yi], 1))

data = create_data.Data(data)
print(len(data))
print([l.index for l in landmarks])

Kmeans...
0
1
2
3
4
5
6
7
8
9
100
[4, 19, 24, 37, 46, 59, 63, 72, 82, 93]


In [6]:
print(type(data))
print(isinstance(data, create_data.Data))

<class 'create_data.Data'>
True


In [17]:
print("Parameter Finding...")

cs = []
pgs = []

for landmark in landmarks:
    c, p_g = bpf.bestParameterFinder(voltage.gaussiankernel, [landmark], data, minBound=3, maxBound=7, granularity=2, epsilon=0.5)
    print(c, p_g)
    cs.append(c)
    pgs.append(p_g)

print(cs)
print(pgs)

Parameter Finding...
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 148.4131591025766 54.598150033144236
5.5 3.5
2.0 148.4131591025766 148.4131591025766
1.0 403.4287934927351 54.598150033144236
6.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 148.4131591025766 54.598150033144236
5.0 3.5
2.0 148.4131591025766 148.4131591025766
1.0 148.4131591025766 54.598150033144236
5.5 3.5
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.5, 6.0, 5.0, 5.5]
[3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.

In [18]:
voltages = []

for index in range(0, len(landmarks)):
    voltages.append(bpf.calculateFor(voltage.gaussiankernel, [landmarks[index]], data, None, cs[index], pgs[index])[0])

In [21]:
bpf.visualizations(voltages, "../inputoutput/matplotfigures/MNIST")

<Figure size 640x480 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [19]:
print(voltages[0])

[1.56650505e-01 1.87745908e-01 2.37758340e-01 2.29309170e-01
 1.00000000e+00 2.93537823e-01 1.04824228e-01 4.10635901e-01
 1.46984615e-01 1.61697570e-01 5.40335964e-04 3.95934541e-04
 4.42730530e-04 6.56298177e-04 4.36297814e-04 3.94677599e-04
 5.72256534e-04 4.17020673e-04 4.70992512e-04 3.83520901e-04
 9.02922437e-03 2.78917517e-03 3.97591795e-03 2.31124529e-03
 3.64428201e-03 2.81744130e-03 2.24540939e-03 2.55807788e-03
 5.19868567e-03 2.15547594e-03 2.10257202e-03 5.53180293e-03
 4.48370253e-03 4.21781563e-03 3.35621569e-03 6.11575964e-03
 6.54057512e-03 4.53181656e-03 4.01193261e-03 7.08640084e-03
 2.09494389e-03 1.28023214e-03 1.19727064e-03 1.21616784e-03
 1.27246000e-03 1.52937714e-03 1.94492934e-03 1.52492795e-03
 1.15032575e-03 1.23359376e-03 1.51423401e-02 3.90734581e-03
 8.94189410e-03 3.08444961e-03 1.35739844e-02 5.94808129e-03
 1.10830378e-02 7.86969527e-03 4.05586395e-03 5.15820574e-03
 1.52104745e-02 6.40036666e-03 5.95837422e-03 7.50893032e-03
 6.46933049e-03 6.068797

In [20]:
print(voltages[1])

[0.00412722 0.00169726 0.00161836 0.00161407 0.00311427 0.00266444
 0.00663559 0.0030662  0.00574318 0.00188665 0.12483099 0.41611854
 0.18187911 0.11127136 0.26307797 0.35641288 0.22952688 0.26030016
 0.16288808 1.         0.00848278 0.0170488  0.00641455 0.01786835
 0.00978387 0.02040137 0.01953977 0.02166803 0.00573706 0.04307813
 0.0206812  0.00564085 0.00924857 0.00885973 0.00944576 0.00683801
 0.00872335 0.01578271 0.00757372 0.0072157  0.00927849 0.01013124
 0.00927517 0.00484809 0.01739856 0.00427721 0.00617114 0.0032118
 0.00772958 0.00621587 0.00811779 0.00893243 0.00649293 0.03322711
 0.00601412 0.0097096  0.00511199 0.01397166 0.01793355 0.01234929
 0.00581219 0.00825552 0.01834228 0.00948554 0.00457998 0.00507363
 0.00387307 0.00791261 0.00510342 0.02147542 0.0205549  0.01495839
 0.00992793 0.00930807 0.00480178 0.00634548 0.00972056 0.0107948
 0.00644844 0.03307827 0.01199526 0.02838957 0.01658443 0.00961939
 0.00879153 0.01150201 0.01207469 0.01461275 0.01304002 0.005568